In [ ]:
import requests
import pandas as pd
import numpy as np
import geopandas as gpd

import shapely.wkt
from pyproj import CRS
import pydeck as pdk
import json 

In [ ]:
mapbox_key = 'pk.eyJ1IjoibGFpcjA4MjYiLCJhIjoiY2tkcGoxcnRzMDZvODJxbXk0MWhlcWN2aSJ9.5-yjt_SUq4w5JII7CvD4cA'

In [ ]:
def add_coordinates(data):
    '''geodataframe에서 중점 coordinates로 컬럼 생성
    '''
    from shapely.geometry import Polygon,MultiLineString,LineString,MultiPolygon
    #coord_center
    temp = data.copy()
    coord_center = temp.geometry.apply(lambda geo : [geo.centroid.x, geo.centroid.y])
    temp['coord_center'] = coord_center
    
    #coordinates type별로 조금씩 차이가 있음
    type_mls = type(MultiLineString())
    type_ls = type(LineString())
    type_poly = type(Polygon())
    type_mpoly = type(MultiPolygon())
    def get_coordinates(x):
        if type(x) == type_mls: 
            lon,lat = x[0].xy
        elif type(x) == type_ls:
            lon,lat = x.xy
        elif type(x) == type_poly:
            lon,lat = x.exterior.xy
        else :
            lon, lat = x[0].exterior.xy        
        return [[x, y] for x, y in zip(lon, lat)]
    
    temp['coordinates'] = temp.geometry.apply(lambda geo : get_coordinates(geo))
    return temp

### 세부링크 도로위험도지수정보 조회 서비스
* input에 링크경로 필요... 어쩌란거지

In [ ]:
key = 'cQ4amAZnPppv3SkKabp8Zo15Jk%2B3S7jcMSaDU1Ej060Hj1xgI%2BEIz1Ov089ktE3bUHIjRC0xyumhChM2I1FhtQ%3D%3D'
url = 'http://apis.data.go.kr/B552061/roadDgdgrLink/getRestRoadDgdgrLink?ServiceKey=서비스키&searchLineString=LineString(127.92136742775328 37.326002556009065, 127.92102024291988 37.32579424030431)&vhctyCd=01&numOfRows=10&pageNo=1'

### 링크기반 사고위험지역정보 조회 서비스 ('epsg:5179')
* 2021-01-11 15:51:31 시점 기준 2019년까지만 지원

In [263]:
key = 'cQ4amAZnPppv3SkKabp8Zo15Jk%2B3S7jcMSaDU1Ej060Hj1xgI%2BEIz1Ov089ktE3bUHIjRC0xyumhChM2I1FhtQ%3D%3D'
year = 2019
sido = 41 #11 -서울
gungu = 370 #680 - 강남
d_type = 'json'
row = 100
page = 1
url = f'http://apis.data.go.kr/B552061/accidentRiskArea/getRestAccidentRiskArea?ServiceKey={key}&searchYearCd={year}&siDo={sido}&guGun={gungu}&numOfRows={row}&pageNo={page}&type={d_type}'

In [264]:
# key = 'cQ4amAZnPppv3SkKabp8Zo15Jk%2B3S7jcMSaDU1Ej060Hj1xgI%2BEIz1Ov089ktE3bUHIjRC0xyumhChM2I1FhtQ%3D%3D'
# year = 2019
# sido = 11 #11 -서울
# gungu = 680 #680 - 강남
# d_type = 'json'
# row = 100
# page = 1
# url = f'http://apis.data.go.kr/B552061/accidentRiskArea/getRestAccidentRiskArea?ServiceKey={key}&searchYearCd={year}&siDo={sido}&guGun={gungu}&numOfRows={row}&pageNo={page}&type={d_type}'

In [265]:
url

'http://apis.data.go.kr/B552061/accidentRiskArea/getRestAccidentRiskArea?ServiceKey=cQ4amAZnPppv3SkKabp8Zo15Jk%2B3S7jcMSaDU1Ej060Hj1xgI%2BEIz1Ov089ktE3bUHIjRC0xyumhChM2I1FhtQ%3D%3D&searchYearCd=2019&siDo=41&guGun=370&numOfRows=100&pageNo=1&type=json'

In [266]:
response = requests.get(url)
response.json()

{'resultCode': '00',
 'resultMsg': 'NORMAL_CODE',
 'items': {'item': [{'acc_risk_area_nm': '경기도 오산시내에서 2019년도에 반경100m 이내 다른사고6건 이상 발생지역',
    'acc_risk_area_id': '59392',
    'geom_wkt': 'POLYGON  (( 962087.78160000 1904013.55440000, 962091.50900000 1904013.92150000, 962110.26830000 1904019.61200000, 962127.55700000 1904028.85300000, 962142.71070000 1904041.28930000, 962155.14700000 1904056.44300000, 962164.38800000 1904073.73170000, 962170.07850000 1904092.49100000, 962172.00000000 1904112.00000000, 962171.80430000 1904113.98740000, 962183.73170000 1904107.61200000, 962202.49100000 1904101.92150000, 962222.00000000 1904100.00000000, 962241.50900000 1904101.92150000, 962260.26830000 1904107.61200000, 962277.55700000 1904116.85300000, 962277.70990000 1904116.97850000, 962284.49100000 1904114.92150000, 962304.00000000 1904113.00000000, 962323.50900000 1904114.92150000, 962328.53780000 1904116.44690000, 962343.50900000 1904117.92150000, 962362.26830000 1904123.61200000, 962379.55700000 19

In [267]:
#응답key값
keys = response.json()['items']['item'][0].keys()

json_data = response.json()['items']['item']
data_dict = {}
for key in keys:
    data_dict[key] = []
for i in range(0, len(json_data)):
    for key in keys:
        value = json_data[i][key]
        data_dict[key].append(value)

temp = pd.DataFrame(data_dict)

#https://stackoverflow.com/questions/51855917/shapely-polygon-from-string
#'문자로 되어있는 폴리곤 변환방법'
temp['geometry'] = temp['geom_wkt'].apply(lambda x : shapely.wkt.loads(x))

In [268]:
temp = gpd.GeoDataFrame(temp)
temp.crs = CRS('epsg:5179')
temp = temp.to_crs('epsg:4326')

temp = add_coordinates(temp)
map_center = temp['coord_center'][0]
temp.head(1)

,acc_risk_area_nm,acc_risk_area_id,geom_wkt,tot_acc_cnt,tot_dth_dnv_cnt,tot_se_dnv_cnt,tot_sl_dnv_cnt,tot_wnd_dnv_cnt,cntpnt_utmk_x_crd,cntpnt_utmk_y_crd,str_cause_anals_ty_nm,cause_anals_ty_nm,geometry,coord_center,coordinates
0,경기도 오산시내에서 2019년도에 반경100m 이내 다른사고6건 이상 발생지역,59392,"POLYGON (( 962087.78160000 1904013.55440000, ...",81,1,24,80,12,962131,1904678,None,"[기타, 안전거리미확보, U턴중]","POLYGON ((127.07316 37.13405, 127.07320 37.134...","[127.07361825258201, 37.14005039082532]","[[127.07315602048648, 37.13405162621939], [127..."


In [ ]:
temp.shape

In [ ]:
sample_layer = pdk.Layer('PolygonLayer', data=temp,
                         get_polygon='coordinates',
                         get_fill_color='[255, 10*tot_acc_cnt, 0, 100]',
                         pickable=True, auto_hightlight=True)
view_states = pdk.ViewState(longitude=map_center[0],
                          latitude = map_center[1],
                           zoom = 12)
r = pdk.Deck(layers=[sample_layer],
             initial_view_state = view_states,
            map_style = 'mapbox://styles/mapbox/outdoors-v11',
            mapbox_key = mapbox_key)
r.show()

### 지자체별사고 다발지역정보 조회 서비스 (EPSG 4326)
* 2021-01-11 16:12:28 기준 2019년까지, 연도별 3건밖에 안나옴..

In [322]:
key = 'cQ4amAZnPppv3SkKabp8Zo15Jk%2B3S7jcMSaDU1Ej060Hj1xgI%2BEIz1Ov089ktE3bUHIjRC0xyumhChM2I1FhtQ%3D%3D'
year = 2018
sido = 41
gungu = 370
d_type = 'json'
n_row= 10
page = 1
url = f'http://apis.data.go.kr/B552061/frequentzoneLg/getRestFrequentzoneLg?ServiceKey={key}&searchYearCd={year}&siDo={sido}&guGun={gungu}&numOfRows={n_row}&pageNo={page}&type={d_type}'

In [323]:
response = requests.get(url)

keys = response.json()['items']['item'][0].keys()
json_data = response.json()['items']['item']
data_dict = {}
for key in keys:
    data_dict[key] = []
for i in range(0, len(json_data)):
    for key in keys:
        value = json_data[i][key]
        data_dict[key].append(value)
temp = pd.DataFrame(data_dict)
# https://ourcstory.tistory.com/106
#string으로 되어있는 dict변환
temp['coordinates'] = temp['geom_json'].apply(lambda x : json.loads(x)['coordinates'])

In [326]:
temp.shape

(3, 16)

In [325]:
sample_layer = pdk.Layer('PolygonLayer', data=temp,
                         get_polygon='coordinates',
                         get_fill_color='[255, 10*tot_acc_cnt, 0, 100]',
                         pickable=True, auto_hightlight=True)
view_states = pdk.ViewState(longitude=float(temp['lo_crd'][0]),
                          latitude = float(temp['la_crd'][1]),
                           zoom = 12)
r = pdk.Deck(layers=[sample_layer],
             initial_view_state = view_states,
            map_style = 'mapbox://styles/mapbox/outdoors-v11',
            mapbox_key = mapbox_key)
r.show()

DeckGLWidget(google_maps_key=None, json_input='{"initialViewState": {"latitude": 37.148972525962, "longitude":…

### 법규위반별사고 다발지역정보 조회 서비스(EPSG 4326)
* 2019년까지
* 4~5건 정도 나옴

In [342]:
key = 'cQ4amAZnPppv3SkKabp8Zo15Jk%2B3S7jcMSaDU1Ej060Hj1xgI%2BEIz1Ov089ktE3bUHIjRC0xyumhChM2I1FhtQ%3D%3D'
year = 2019
sido = 41
gungu = 370
d_type = 'json'
n_row= 10
page = 1
url = f'http://apis.data.go.kr/B552061/frequentzoneLgrViolt/getRestFrequentzoneLgrViolt?ServiceKey={key}&searchYearCd={year}&siDo={sido}&guGun={gungu}&numOfRows={n_row}&pageNo={page}&type={d_type}'

In [349]:
response.json()['totalCount']

4

In [343]:
response = requests.get(url)

keys = response.json()['items']['item'][0].keys()
json_data = response.json()['items']['item']
data_dict = {}
for key in keys:
    data_dict[key] = []
for i in range(0, len(json_data)):
    for key in keys:
        value = json_data[i][key]
        data_dict[key].append(value)
temp = pd.DataFrame(data_dict)
# https://ourcstory.tistory.com/106
#string으로 되어있는 dict변환
temp['coordinates'] = temp['geom_json'].apply(lambda x : json.loads(x)['coordinates'])

In [344]:
temp.shape

(4, 16)

In [345]:
sample_layer = pdk.Layer('PolygonLayer', data=temp,
                         get_polygon='coordinates',
                         get_fill_color='[255, 10*tot_acc_cnt, 0, 100]',
                         pickable=True, auto_hightlight=True)
view_states = pdk.ViewState(longitude=float(temp['lo_crd'][0]),
                          latitude = float(temp['la_crd'][1]),
                           zoom = 12)
r = pdk.Deck(layers=[sample_layer],
             initial_view_state = view_states,
            map_style = 'mapbox://styles/mapbox/outdoors-v11',
            mapbox_key = mapbox_key)
r.show()

DeckGLWidget(google_maps_key=None, json_input='{"initialViewState": {"latitude": 37.189800309737, "longitude":…

### 보행어린이사고다발지역정보서비스
* 2015년까지만 있음
* 1건정도씩밖에 없음

In [382]:
key = 'cQ4amAZnPppv3SkKabp8Zo15Jk%2B3S7jcMSaDU1Ej060Hj1xgI%2BEIz1Ov089ktE3bUHIjRC0xyumhChM2I1FhtQ%3D%3D'
year = 2014
sido = 41
gungu = 370
d_type = 'json'
n_row= 10
page = 1
url = f'http://apis.data.go.kr/B552061/frequentzoneChild/getRestFrequentzoneChild?serviceKey={key}&searchYearCd={year}&siDo={sido}&guGun={gungu}&type={d_type}&numOfRows={n_row}&pageNo={page}'

In [383]:
response.json()['totalCount']

1

In [376]:
response = requests.get(url)

keys = response.json()['items']['item'][0].keys()
json_data = response.json()['items']['item']
data_dict = {}
for key in keys:
    data_dict[key] = []
for i in range(0, len(json_data)):
    for key in keys:
        value = json_data[i][key]
        data_dict[key].append(value)
temp = pd.DataFrame(data_dict)
# https://ourcstory.tistory.com/106
#string으로 되어있는 dict변환
temp['coordinates'] = temp['geom_json'].apply(lambda x : json.loads(x)['coordinates'])

In [377]:
temp.shape

(1, 16)

In [379]:
temp

,afos_fid,afos_id,bjd_cd,spot_cd,sido_sgg_nm,spot_nm,occrrnc_cnt,caslt_cnt,dth_dnv_cnt,se_dnv_cnt,sl_dnv_cnt,wnd_dnv_cnt,geom_json,lo_crd,la_crd,coordinates
0,6165531,2016044,4137010300,41370001,경기도 오산시1,경기도 오산시 원동((거산농수산마트앞교차로) 부근),3,3,0,3,0,0,"{""type"":""Polygon"",""coordinates"":[[[127.0708252...",127.069028648854,37.133680143625,"[[[127.07082528, 37.13368014], [127.07079076, ..."


In [380]:
sample_layer = pdk.Layer('PolygonLayer', data=temp,
                         get_polygon='coordinates',
                         get_fill_color='[255, 10*tot_acc_cnt, 0, 100]',
                         pickable=True, auto_hightlight=True)
view_states = pdk.ViewState(longitude=float(temp['lo_crd'][0]),
                          latitude = float(temp['la_crd'][0]),
                           zoom = 12)
r = pdk.Deck(layers=[sample_layer],
             initial_view_state = view_states,
            map_style = 'mapbox://styles/mapbox/outdoors-v11',
            mapbox_key = mapbox_key)
r.show()

DeckGLWidget(google_maps_key=None, json_input='{"initialViewState": {"latitude": 37.133680143625, "longitude":…

### 스쿨존내 어린이 사고 다발지 현황-경기데이터드림
* 2014년 하나 있음

In [393]:
key = '32e846a7349d462b9c2c1c86a7386617'
page = 1
n_row = 100
name_sido = '오산시'
url = f'https://openapi.gg.go.kr/Tbtninsttm?KEY={key}&Type=json&pindex={page}&pSize={n_row}&SIGUN_NM={name_sido}'

In [394]:
response = requests.get(url)

In [395]:
response

<Response [200]>

### 보행 어린이 사고 다발지 현황-경기데이터드림
* 2015년 하나 있음. 젠장

In [397]:
key = '32e846a7349d462b9c2c1c86a7386617'
page = 1
n_row = 100
name_sido = '오산시'
url = f'https://openapi.gg.go.kr/TbPdstrnchildTfcacdarM?KEY={key}&Type=json&pindex={page}&pSize={n_row}&SIGUN_NM={name_sido}'

In [398]:
response = requests.get(url)

In [403]:
response.json()

{'TbPdstrnchildTfcacdarM': [{'head': [{'list_total_count': 1},
    {'RESULT': {'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다.'}},
    {'api_version': '1.0'}]},
  {'row': [{'SIGUN_NM': '오산시',
     'SIGUN_CD': '41370',
     'EMD_NM': '원동',
     'ACDNT_DIV_NM': '보행어린이사고다발지',
     'MULTI_KNOWLG_DIV_NO': 6165531,
     'MULTI_KNOWLG_DIV_GROUP_NO': 2016044,
     'LEGALDONG_CD_NO': 4137010300,
     'SPOT_NO': '41370001',
     'JURISD_POLCSTTN_NM': '경기도 오산시1',
     'LOC_INFO': '경기도 오산시 원동((거산농수산마트앞교차로) 부근)',
     'OCCUR_CNT': 3,
     'CASLT_CNT': 3,
     'DPRS_CNT': 0,
     'SERINJRY_INDVDL_CNT': 3,
     'SLTINJRY_INDVDL_CNT': 0,
     'INJURY_APLCNT_CNT': 0,
     'LOGT': '127.069028648854',
     'LAT': '37.133680143625',
     'MULTI_REGION_INFO': '{"type":"Polygon","coordinates":[[[127.07082528,37.13368014],[127.07079076,37.13340071],[127.07068852,37.13313201],[127.07052249,37.13288438],[127.07029906,37.13266733],[127.0700268,37.1324892],[127.06971619,37.13235684],[127.06937915,37.13227533],[127.0

### 교통사고정보 개방시스템
* 왜인지 한건씩밖에 안나온다
http://taas.koroad.or.kr/api/selectChildDataSet.do#

In [428]:
key = 'TslM3oDqkki6hTTsjl%2Bl0behLzNwoW3vj1Xxhj0aOaGPK31Qd7NbeMt6%2F1QrigT4'
year = 2019
sido = 11
gungu = 305
d_type = 'json'
n_row= 10
page = 1
url = f'http://taas.koroad.or.kr/data/rest/frequentzone/child?authKey={key}&searchYearCd={year}&siDo={sido}&guGun={gungu}&type={d_type}&numOfRows={n_row}&pageNo={page}'
# url = f'http://taas.koroad.or.kr/data/rest/frequentzone/child?authKey={key}'

In [429]:
url

'http://taas.koroad.or.kr/data/rest/frequentzone/child?authKey=TslM3oDqkki6hTTsjl%2Bl0behLzNwoW3vj1Xxhj0aOaGPK31Qd7NbeMt6%2F1QrigT4&searchYearCd=2019&siDo=11&guGun=305&type=json&numOfRows=10&pageNo=1'

In [430]:
response = requests.get(url)

In [431]:
response.json()

{'resultCode': '00',
 'resultMsg': 'NORMAL_CODE',
 'items': {'item': [{'afos_fid': 6597527,
    'afos_id': '2020016',
    'bjd_cd': '1130510200',
    'spot_cd': '11305001',
    'sido_sgg_nm': '서울특별시 강북구1',
    'spot_nm': '서울특별시 강북구 번동(삼덕상운 부근)',
    'occrrnc_cnt': 4,
    'caslt_cnt': 4,
    'dth_dnv_cnt': 0,
    'se_dnv_cnt': 0,
    'sl_dnv_cnt': 4,
    'wnd_dnv_cnt': 0,
    'geom_json': '{"type":"Polygon","coordinates":[[[127.04331451,37.62801501],[127.04327999,37.62773742],[127.04317775,37.62747048],[127.04301173,37.62722448],[127.04278829,37.62700885],[127.04251604,37.62683189],[127.04220542,37.6267004],[127.04186839,37.62661942],[127.04151788,37.62659208],[127.04116738,37.62661942],[127.04083034,37.6267004],[127.04051973,37.62683189],[127.04024747,37.62700885],[127.04002404,37.62722448],[127.03985801,37.62747048],[127.03975578,37.62773742],[127.03972125,37.62801501],[127.03975578,37.62829261],[127.03985801,37.62855954],[127.04002404,37.62880554],[127.04024747,37.62902116],[127.0405